<a href="https://colab.research.google.com/github/MrDarkTesla/QCL_2021/blob/main/QCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install qiskit

In [ ]:
# Initialization
import matplotlib.pyplot as plt
import numpy as np

# Importing Qiskit
from qiskit import IBMQ, BasicAer, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# Import basic plot tools
from qiskit.tools.visualization import plot_histogram

# Scipy
from scipy.optimize import rosen, differential_evolution

# Files
from google.colab import files

In [ ]:
def f1(x):
  return x ** 2 + 1


def f2(x):
  return x ** 2 + x ** 3


def f3(x):
  return np.sin(x + 1)


def f4(x):
  return np.exp(x) + x


TEACHER = np.random.choice(np.linspace(-1, 1), 10)

In [ ]:
def U_in(x, xq, qc):
  qc.rz(np.arccos(x ** 2), xq)
  qc.ry(np.arcsin(x), xq)


def U_out(i, j, xq, qc, theta, n):
  t = (i + j * n) * 3
  qc.rx(theta[t + 0], xq[i])
  qc.rz(theta[t + 1], xq[i])
  qc.rx(theta[t + 2], xq[i])


def Hamelton(n, qc):
  for i in range(n - 1):
    qc.cx(i, i + 1)
#  for i in range(n - 1, 0, -1):
#    qc.cx(i, i - 1)


def Bfunk(n, iter, count):
  b = [0] * n
  for k in count.keys():
    for i, l in enumerate(k):
      b[i] += int(l) * count[k]
  for i in range(n):
    b[i] /= iter
  return b



In [ ]:
def fi(xi, N, deapth, theta, Iterations, Omega):

  xq = QuantumRegister(N, "x")
  yc = ClassicalRegister(N, "y")
  qc = QuantumCircuit(xq, yc)

  U_in(xi, xq, qc) 
  for j in range(deapth):
    Hamelton(N, qc)
    for k in range(N):
      U_out(k, j, xq, qc, theta, N)

  qc.measure(xq, yc)
  job = execute(qc, backend, shots=Iterations)
  result = job.result()
  count = result.get_counts()
  B = Bfunk(N, Iterations, count)
  Y = np.dot(B, Omega)
  return Y

In [ ]:
def Lfunk(Theta):
  Omega = [1/3, 1/3, 1/3]
  Iterations = 1000 
  deapth = 2   
  N = 3 
  L = np.dot(Omega, Omega) 
  for t in TEACHER:
    L += (f1(t) - fi(t, N, deapth, theta, Iterations, Omega)) ** 2
  return L

In [ ]:
theta = np.zeros(3 * 2 * 3)
bounds = [(0,2 * np.pi)] * len(theta)

backend = Aer.get_backend('qasm_simulator')
result = differential_evolution(Lfunk, bounds)
np.save("Result.npy", result)
files.download('Test.npy')

RuntimeError: ignored